In [1]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from utils.plot import plot_losses, plot_train_val_ap_ars
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.datasets import  OurRadiologsitsDataset, collate_fn
from our_radiologist.load import get_anns
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ar_ap
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.constants import full_iou_thrs, iou_thrs_5to95
from data.load  import seed_worker, get_dataloader_g
import PIL
from matplotlib.figure import Figure
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn

from typing import Callable, Dict, List, Union, Tuple
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from matplotlib import colors
from utils.pred import pred_thrs_check
from utils.save import get_data_from_metric_logger
from data.datasets import ReflacxDataset, collate_fn
from utils.detect_utils import MetricLogger
from utils.coco_eval import CocoEvaluator, external_summarize
from utils.plot import DISEASE_CMAP, get_legend_elements
from models.train import TrainingInfo


## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline
# plt.ioff()

c:\Users\mike8\Desktop\multimodal-abnormalities-detection\models\detectors\rcnn.py:832: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"
labels_cols = DEFAULT_REFLACX_LABEL_COLS
iou_thrs = np.array([0.5])


In [4]:
from enum import Enum

class TrainedModels(Enum):

    CXR_Clinical_ap = "val_ar_0_9381_ap_0_6945_test_ar_0_8224_ap_0_6378_epoch20_WithClincal_05-16-2022 18-13-10_CXR+Clinical"
    CXR_Clinical_final = "val_ar_0_9556_ap_0_6776_test_ar_0_7849_ap_0_6534_epoch71_WithClincal_05-16-2022 19-38-03_CXR+Clinical"

    CXR_ap = "val_ar_0_7518_ap_0_5042_test_ar_0_5388_ap_0_3255_epoch22_WithoutClincal_05-16-2022 20-18-25_CXR"
    CXR_final = "val_ar_0_5966_ap_0_4046_test_ar_0_5388_ap_0_3893_epoch54_WithoutClincal_05-16-2022 21-10-18_CXR"

In [5]:
select_model = TrainedModels.CXR_final

model, train_info, _ = get_trained_model(
    select_model,
    DEFAULT_REFLACX_LABEL_COLS,
    device,
    rpn_nms_thresh=0.3,
    box_detections_per_img=10,
    box_nms_thresh=0.2,
    rpn_score_thresh=0.0,
    box_score_thresh=0.05,
)

model.to(device)
model.eval()


Load custom model
Using pretrained backbone. mobilenet_v3
Found optimizer for this model.
Using SGD as optimizer with lr=0.001


MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
    

In [6]:
dataset_params_dict = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "with_clinical": True,
    "dataset_mode": 'unified',
    "bbox_to_mask": False,
    "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
}

detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
    dataset_params_dict=dataset_params_dict,
)

train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size=4,
)

test_coco = get_coco_api_from_dataset(test_dataloader.dataset)

creating index...
index created!


In [7]:
eval_params_dict = get_eval_params_dict(
    detect_eval_dataset, iou_thrs=iou_thrs, use_iobb=use_iobb,
)

iou_types = get_iou_types(model, train_info.model_setup)

creating index...
index created!


In [8]:
outputs, data = xami_evaluate(
            model=model,
            data_loader=test_dataloader,
            device=device,
            params_dict=eval_params_dict,
            coco=test_coco,
            iou_types=iou_types,
            score_thres=None,
        )

In [9]:
outputs

[{'boxes': tensor([[1596.4518,  835.9207, 2544.0000, 2298.1069]]),
  'labels': tensor([5]),
  'scores': tensor([0.0950])},
 {'boxes': tensor([[ 948.7343, 1526.3390, 1952.4445, 1944.0891],
          [ 816.2150, 1605.5148, 1327.4039, 2301.0000],
          [1893.3350, 1311.7275, 2417.0811, 2125.7214],
          [ 109.7983, 1391.6091,  944.9073, 1963.8418],
          [   0.0000, 1417.2985, 1484.9883, 2298.9753],
          [ 888.6131, 1582.9867, 1258.2063, 1868.1633],
          [ 461.8488, 1013.4288,  977.7272, 1854.6113],
          [   0.0000, 1449.7666, 1376.3319, 2301.0000]]),
  'labels': tensor([1, 2, 2, 3, 4, 4, 4, 5]),
  'scores': tensor([0.8455, 0.6200, 0.3789, 0.2310, 0.1887, 0.1779, 0.1017, 0.0704])},
 {'boxes': tensor([], size=(0, 4)),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([])},
 {'boxes': tensor([[ 393.4535, 1746.0244, 2170.3828, 2847.0959],
          [ 100.2309, 2156.9573,  641.9927, 2729.5427],
          [ 405.5574, 1394.8773,  868.5236, 2439.0625],
    

In [10]:
model.eval()
images, clinical_num, clinical_cat, targets = data

In [11]:
original_image_sizes: List[Tuple[int, int]] = []
for img in images:
    val = img.shape[-2:]
    assert len(val) == 2
    original_image_sizes.append((val[0], val[1]))

In [12]:
images, targets = model.transform(images, targets)

In [13]:
from collections import OrderedDict

img_features = model.backbone(images.tensors)
if isinstance(img_features, torch.Tensor):
    img_features = OrderedDict([("0", img_features)])


In [14]:
# clinical_features = model.get_clinical_features(clinical_num, clinical_cat)
# features = model.fuse_features(img_features, clinical_features)

In [16]:
features = img_features

In [34]:
proposals, proposal_losses = model.rpn(images, features, targets)

In [36]:
for p in proposals:
    print(p.shape)


# Without
# torch.Size([113, 4])
# torch.Size([111, 4])
# torch.Size([115, 4])
# torch.Size([110, 4])

# With
# torch.Size([113, 4])
# torch.Size([111, 4])
# torch.Size([115, 4])
# torch.Size([110, 4])

# Same

torch.Size([113, 4])
torch.Size([111, 4])
torch.Size([115, 4])
torch.Size([110, 4])


In [37]:
# detections, detector_losses = model.roi_heads(
#     features, proposals, images.image_sizes, None
# )

In [38]:
image_shapes = images.image_sizes

In [43]:
t = targets
if t:
    (
        proposals,
        matched_idxs,
        labels,
        regression_targets,
    ) = model.roi_heads.select_training_samples(
        proposals, targets
    )
# else:
# labels = None
# regression_targets = None
# matched_idxs = None

In [45]:
for p in proposals:
    print(p.shape)

# Without
# torch.Size([113, 4])
# torch.Size([111, 4])
# torch.Size([115, 4])
# torch.Size([110, 4])

# With
# torch.Size([113, 4])
# torch.Size([113, 4]) + 2
# torch.Size([115, 4])
# torch.Size([112, 4]) + 2

## the ground truth is appended.

torch.Size([113, 4])
torch.Size([113, 4])
torch.Size([115, 4])
torch.Size([112, 4])


In [47]:
for t in targets:
    print(len(t['boxes']))

0
2
0
2


In [178]:
targets

[{'image_path': 'D:\\XAMI-MIMIC\\patient_19875621\\CXR-JPG\\s55102074\\a2fe8aae-2fe32131-b47c4e5b-090f4c13-88e7ac97.jpg',
  'dicom_id': 'a2fe8aae-2fe32131-b47c4e5b-090f4c13-88e7ac97',
  'iscrowd': tensor([], device='cuda:0', dtype=torch.int64),
  'area': tensor([], device='cuda:0', dtype=torch.float64),
  'image_id': tensor([57], device='cuda:0'),
  'labels': tensor([], device='cuda:0', dtype=torch.int64),
  'boxes': tensor([], device='cuda:0', size=(0, 4), dtype=torch.float64)},
 {'image_path': 'D:\\XAMI-MIMIC\\patient_19472857\\CXR-JPG\\s59311723\\494d5cdc-f7b4d476-5b2a9c62-fb75fed7-4514dd92.jpg',
  'dicom_id': '494d5cdc-f7b4d476-5b2a9c62-fb75fed7-4514dd92',
  'iscrowd': tensor([0, 0], device='cuda:0'),
  'area': tensor([477660., 257640.], device='cuda:0', dtype=torch.float64),
  'image_id': tensor([55], device='cuda:0'),
  'labels': tensor([4, 3], device='cuda:0'),
  'boxes': tensor([[  9.6570, 155.2021,  84.5882, 218.6180],
          [  5.2756, 178.7883,  56.2431, 229.0760]], devic

In [102]:
box_features = model.roi_heads.box_roi_pool(features, proposals, image_shapes)
box_features = model.roi_heads.box_head(box_features)
class_logits, box_regression = model.roi_heads.box_predictor(box_features)
pred_boxes, pred_scores, pred_labels = model.roi_heads.postprocess_detections(
    class_logits, box_regression, proposals, image_shapes
)


In [103]:
pred_boxes

[tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([[  5.4673, 179.0961,  56.1635, 229.1518],
         [  9.5961, 155.1163,  84.5731, 218.5875]], device='cuda:0',
        grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([[  9.9202, 181.0386,  64.4764, 228.9503],
         [ 40.2115,  98.6401,  98.7625, 199.0232],
         [ 40.1573,  98.5876,  98.5721, 199.1081],
         [ 40.1496,  98.6221,  98.0900, 198.9981],
         [  9.9747, 180.6959,  64.9986, 228.6949],
         [  9.8982, 180.8730,  65.0026, 229.4350],
         [ 40.0709,  98.6745,  98.3318, 198.7289],
         [ 40.2721,  98.6023,  98.6496, 199.1269]], device='cuda:0',
        grad_fn=<IndexBackward0>)]

In [104]:
proposals

[tensor([[  0.0000,   0.0000,  64.0057,  63.9007],
         [  0.0000,   0.0000,  16.0167,  15.9914],
         [  0.0000,   0.0000,  90.9638, 181.0567],
         [  0.0000,   0.0000,  45.1015,  22.9766],
         [  0.0000,   0.0000, 255.8519, 255.8518]], device='cuda:0'),
 tensor([[  0.0000,   0.0000,  91.1129,  45.1633],
         [  0.0000,   0.0000,  16.0130,  15.9975],
         [  0.0000,   0.0000, 181.1748,  91.1633],
         [  0.0000,   0.0000,  23.0024,  45.0016],
         [  0.0000,   0.0000, 255.7023, 256.0000],
         [  9.6570, 155.2021,  84.5882, 218.6180],
         [  5.2756, 178.7883,  56.2431, 229.0760]], device='cuda:0'),
 tensor([[  0.0000,   0.0000,  63.9729,  63.8942],
         [  0.0000,   0.0000,  16.0227,  15.9878],
         [  0.0000,   0.0000,  90.9613, 181.0904],
         [  0.0000,   0.0000,  45.1665,  22.9645],
         [  0.0000,   0.0000, 255.9640, 255.8592]], device='cuda:0'),
 tensor([[  0.0000,   0.0000,  64.0149,  63.9659],
         [  0.0000,   0.0

In [108]:
proposals


[tensor([[  0.0000,   0.0000,  64.0057,  63.9007],
         [  0.0000,   0.0000,  16.0167,  15.9914],
         [  0.0000,   0.0000,  90.9638, 181.0567],
         [  0.0000,   0.0000,  45.1015,  22.9766],
         [  0.0000,   0.0000, 255.8519, 255.8518]], device='cuda:0'),
 tensor([[  0.0000,   0.0000,  91.1129,  45.1633],
         [  0.0000,   0.0000,  16.0130,  15.9975],
         [  0.0000,   0.0000, 181.1748,  91.1633],
         [  0.0000,   0.0000,  23.0024,  45.0016],
         [  0.0000,   0.0000, 255.7023, 256.0000],
         [  9.6570, 155.2021,  84.5882, 218.6180],
         [  5.2756, 178.7883,  56.2431, 229.0760],
         [  9.6570, 155.2021,  84.5882, 218.6180],
         [  5.2756, 178.7883,  56.2431, 229.0760]], device='cuda:0'),
 tensor([[  0.0000,   0.0000,  63.9729,  63.8942],
         [  0.0000,   0.0000,  16.0227,  15.9878],
         [  0.0000,   0.0000,  90.9613, 181.0904],
         [  0.0000,   0.0000,  45.1665,  22.9645],
         [  0.0000,   0.0000, 255.9640, 255.

In [122]:
proposals

[tensor([[ 79.7310,  80.0415, 111.2964, 112.1849],
         [ 79.5596,  48.2780, 111.1421,  80.2240],
         [ 79.1171, 175.6689, 110.9923, 207.6157],
         [ 79.6397, 112.3669, 111.4200, 144.2354],
         [143.7118, 175.9557, 175.5651, 207.6567],
         [  1.2675, 116.4223, 187.1969, 206.6625],
         [111.3638, 175.7640, 143.3059, 207.6096],
         [ 47.3699,  47.4726,  79.2469,  79.5155],
         [175.7124,  47.8465, 207.5594,  80.3272],
         [  3.0382,   0.0000, 188.5395,  47.3403],
         [ 95.0514,  33.9334, 223.9072, 163.8932],
         [175.7179,  80.0144, 207.4230, 112.3964],
         [ 82.5983, 105.3650, 173.0258, 151.6001],
         [175.9001, 112.5397, 207.9298, 145.0254],
         [147.6545, 105.4588, 237.8342, 151.2391],
         [143.4365,  79.0752, 175.3068, 111.3377],
         [175.0590,  16.1108, 207.1270,  48.0714],
         [ 15.5587, 175.6856,  47.4360, 207.8766],
         [ 97.0135,   0.0000, 256.0000,  79.3423],
         [ 47.8205,  79.8412,  